<a href="https://colab.research.google.com/github/Sharondafrin/FB-Likes-Prediction/blob/main/Table_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
#import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import xml.etree.ElementTree as ET
from PIL import Image

#tensorflow libraries
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input, Concatenate, UpSampling2D

In [14]:
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)

Mounted at /content/drive


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
#length of bmp files we have
len(glob.glob('/content/drive/MyDrive/Colab Notebooks/dataset/Marmot_data/*.bmp'))

494

In [17]:
#length of xml files we have
len(glob.glob('/content/drive/MyDrive/Colab Notebooks/dataset/Marmot_data/*.xml'))

494

In [18]:
#changing bmp image to jpg
IMAGE_PATH = '/content/drive/MyDrive/Colab Notebooks/dataset/Marmot_data'
for image in glob.glob('/content/drive/MyDrive/Colab Notebooks/dataset/Marmot_data/*.bmp'):
    im = Image.open(image)
    image_name = os.path.basename(image)
    save_image_path = os.path.join(IMAGE_PATH, image_name.replace('bmp', 'jpg'))
    im.save(save_image_path)

In [19]:
len(glob.glob('/content/drive/MyDrive/Colab Notebooks/dataset/Marmot_data/*.jpg'))

494

## Data Preperation

In [ ]:
def sameTable(ymin_1, ymin_2, ymax_1, ymax_2):
    min_diff = abs(ymin_1 - ymin_2)
    max_diff = abs(ymax_1 - ymax_2)

    if min_diff <= 5 and max_diff <=5:
        return True
    elif min_diff <= 4 and max_diff <=7:
        return True
    elif min_diff <= 7 and max_diff <=4:
        return True
    return False

In [ ]:
if __name__ == "__main__":
    directory = r'/content/drive/MyDrive/Colab Notebooks/dataset/Marmot_data'
    final_col_directory = r'/content/drive/MyDrive/Colab Notebooks/dataset/Columns Mask'
    final_table_directory = r'/content/drive/MyDrive/Colab Notebooks/dataset/Table Mask'

    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        # Find all the xml files
        if filename.endswith(".xml"):
            filename = filename[:-4]

            # Parse xml file
            tree = ET.parse(directory + '\\' + filename + '.xml')
            root = tree.getroot()
            size = root.find('size')
            
            # Parse width
            width = int(size.find('width').text)
            height = int(size.find('height').text)

            # Create grayscale image array
            col_mask = np.zeros((height, width), dtype=np.int32)
            table_mask = np.zeros((height, width), dtype = np.int32)

            got_first_column = False
            i=0
            table_xmin = 10000
            table_xmax = 0

            table_ymin = 10000
            table_ymax = 0

            for column in root.findall('object'):
                bndbox = column.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)

                col_mask[ymin:ymax, xmin:xmax] = 255

                if got_first_column:
                    if sameTable(prev_ymin, ymin, prev_ymax, ymax) == False:
                        i+=1
                        got_first_column = False
                        table_mask[table_ymin:table_ymax, table_xmin:table_xmax] = 255

                        table_xmin = 10000
                        table_xmax = 0

                        table_ymin = 10000
                        table_ymax = 0

                if got_first_column == False:
                    got_first_column = True
                    first_xmin = xmin

                prev_ymin = ymin
                prev_ymax = ymax

                table_xmin = min(xmin, table_xmin)
                table_xmax = max(xmax, table_xmax)

                table_ymin = min(ymin, table_ymin)
                table_ymax = max(ymax, table_ymax)

            table_mask[table_ymin:table_ymax, table_xmin:table_xmax] = 255

            im = Image.fromarray(col_mask.astype(np.uint8),'L')
            im.save(final_col_directory + "\\" + filename + ".jpeg")

            im = Image.fromarray(table_mask.astype(np.uint8),'L')
            im.save(final_table_directory + "\\" + filename + ".jpeg")

FileNotFoundError: ignored

## Training the data

In [20]:
img_height, img_width = 256, 256
def normalize(input_image):

    input_image = tf.cast(input_image, tf.float32) / 255.0

    return input_image

def decode_img(img):

    img = tf.image.decode_jpeg(img)

    return tf.image.resize(img, [img_height, img_width])

def decode_mask_img(img):

    img = tf.image.decode_jpeg(img, channels=1)

    return tf.image.resize(img, [img_height, img_width])

def create_mask(pred_mask1, pred_mask2):

    pred_mask1 = tf.argmax(pred_mask1, axis=-1)
    pred_mask1 = pred_mask1[..., tf.newaxis]
    pred_mask2 = tf.argmax(pred_mask2, axis=-1)
    pred_mask2 = pred_mask2[..., tf.newaxis]

    return pred_mask1[0], pred_mask2[0]

def display(display_list):

    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'Table Mask', 'Column Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
        plt.show()

    return None

def process_path(file_path):

    file_path = tf.strings.regex_replace(file_path, '.xml', '.jpg')
    #print(file_path)
    mask_file_path = tf.strings.regex_replace(file_path, '.jpg', '.jpeg')
    print(mask_file_path)
    table_mask_file_path = tf.strings.regex_replace(mask_file_path, 'Marmot_data', 'Table Mask')
    column_mask_file_path = tf.strings.regex_replace(mask_file_path, 'Marmot_data', 'Columns Mask')
    img = normalize(decode_img(tf.io.read_file(file_path)))
    table_mask = normalize(decode_mask_img(tf.io.read_file(table_mask_file_path)))
    column_mask = normalize(decode_mask_img(tf.io.read_file(column_mask_file_path)))

    return img, {"table_output" : table_mask, "column_output" : column_mask}

class TableNet:

    @staticmethod
    def build_table_decoder(inputs, pool3, pool4):
        x = Conv2D(512, (1, 1), activation = 'relu', name='conv7_table')(inputs)
        x = UpSampling2D(size=(2, 2))(x)

        concatenated = Concatenate()([x, pool4])

        x = UpSampling2D(size=(2,2))(concatenated)

        concatenated = Concatenate()([x, pool3])

        x = UpSampling2D(size=(2,2))(concatenated)
        x = UpSampling2D(size=(2,2))(x)

        last = tf.keras.layers.Conv2DTranspose(3, 3, strides=2, padding='same', name='table_output')

        x = last(x)

        return x

    @staticmethod
    def build_column_decoder(inputs, pool3, pool4):

        x = Conv2D(512, (1, 1), activation = 'relu', name='block7_conv1_column')(inputs)
        x = Dropout(0.8, name='block7_dropout_column')(x)

        x = Conv2D(512, (1, 1), activation = 'relu', name='block8_conv1_column')(x)
        x = UpSampling2D(size=(2, 2))(x)

        concatenated = Concatenate()([x, pool4])

        x = UpSampling2D(size=(2,2))(concatenated)

        concatenated = Concatenate()([x, pool3])

        x = UpSampling2D(size=(2,2))(concatenated)
        x = UpSampling2D(size=(2,2))(x)

        last = tf.keras.layers.Conv2DTranspose(3, 3, strides=2, padding='same', name='column_output')

        x = last(x)

        return x

    @staticmethod
    def vgg_base(inputs):
        base_model = tf.keras.applications.vgg19.VGG19(
            input_shape=[256, 256, 3],
            include_top=False, weights='imagenet')

        layer_names = ['block3_pool', 'block4_pool', 'block5_pool']
        layers = [base_model.get_layer(name).output for name in layer_names]

        pool_layers_model = Model(inputs=base_model.input, outputs=layers, name='VGG-19')
        pool_layers_model.trainable = False

        return pool_layers_model(inputs)

    @staticmethod
    def build():
        inputShape = (256, 256, 3)

        inputs = Input(shape=inputShape, name='input')

        pool_layers = TableNet.vgg_base(inputs)

        x = Conv2D(512, (1, 1), activation = 'relu', name='block6_conv1')(pool_layers[2])
        x = Dropout(0.8, name='block6_dropout1')(x)
        x = Conv2D(512, (1, 1), activation = 'relu', name='block6_conv2')(x)
        x = Dropout(0.8, name = 'block6_dropout2')(x)

        table_mask = TableNet.build_table_decoder(x, pool_layers[0], pool_layers[1])
        column_mask = TableNet.build_column_decoder(x, pool_layers[0], pool_layers[1])

        model = Model(
                        inputs=inputs,
                        outputs=[table_mask, column_mask],
                        name="tablenet")

        return model

def main():

    list_ds = tf.data.Dataset.list_files(os.path.join(train_data,'*xml'), shuffle=False)

    DATASET_SIZE = len(list(list_ds))
    train_size = int(0.9 * DATASET_SIZE)
    test_size = int(0.1 * DATASET_SIZE)

    train = list_ds.take(train_size)
    test = list_ds.skip(train_size)

    TRAIN_LENGTH = len(list(train))
    BATCH_SIZE = 1
    BUFFER_SIZE = 10
    STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

    train = train.shuffle(BUFFER_SIZE)

    train = train.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    test = test.map(process_path)

    train_dataset = train.batch(BATCH_SIZE).repeat().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    test_dataset = test.batch(BATCH_SIZE)

    losses = {
        "table_output": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        "column_output": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    }

    lossWeights = {"table_output": 1.0, "column_output": 1.0}

    model = TableNet.build()

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, epsilon=1e-08),
                  loss=losses,
                  metrics=['accuracy'],
                  loss_weights=lossWeights)

    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
            filepath="/content/drive/MyDrive/Colab Notebooks/Model_output/mymodel_{epoch}",
            save_best_only=True,
            monitor="val_loss",
            verbose=1,
        )

    early_stoping = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=5,
            verbose=1,
        )

    EPOCHS = 400
    VAL_SUBSPLITS = 5
    VALIDATION_STEPS = test_size//BATCH_SIZE//VAL_SUBSPLITS


    model_history = model.fit(train_dataset, epochs=EPOCHS,
                              steps_per_epoch=STEPS_PER_EPOCH,
                              validation_steps=VALIDATION_STEPS,
                              validation_data=test_dataset,
                              callbacks=[model_checkpoint])

    model.save('/content/drive/MyDrive/Colab Notebooks/Model_output/model_tes')

if __name__ == "__main__":

    train_data = '/content/drive/MyDrive/Colab Notebooks/dataset/Marmot_data'

    main()


Tensor("StaticRegexReplace_1:0", shape=(), dtype=string)
Tensor("StaticRegexReplace_1:0", shape=(), dtype=string)
Epoch 1/400
444/444 [==============================] - ETA: 0s - loss: 0.9163 - table_output_loss: 0.5206 - column_output_loss: 0.3957 - table_output_accuracy: 0.8453 - column_output_accuracy: 0.8592
Epoch 1: val_loss improved from inf to 0.69599, saving model to /content/drive/MyDrive/Colab Notebooks/Model_output/mymodel_1


444/444 [==============================] - 24s 52ms/step - loss: 0.9163 - table_output_loss: 0.5206 - column_output_loss: 0.3957 - table_output_accuracy: 0.8453 - column_output_accuracy: 0.8592 - val_loss: 0.6960 - val_table_output_loss: 0.3770 - val_column_output_loss: 0.3189 - val_table_output_accuracy: 0.8532 - val_column_output_accuracy: 0.8646
Epoch 2/400
443/444 [============================>.] - ETA: 0s - loss: 0.5085 - table_output_loss: 0.2582 - column_output_loss: 0.2502 - table_output_accuracy: 0.8976 - column_output_accuracy: 0.8849
Epoch 2: val_loss improved from 0.69599 to 0.49438, saving model to /content/drive/MyDrive/Colab Notebooks/Model_output/mymodel_2


444/444 [==============================] - 22s 49ms/step - loss: 0.5080 - table_output_loss: 0.2580 - column_output_loss: 0.2500 - table_output_accuracy: 0.8977 - column_output_accuracy: 0.8850 - val_loss: 0.4944 - val_table_output_loss: 0.2259 - val_column_output_loss: 0.2685 - val_table_output_accuracy: 0.9086 - val_column_output_accuracy: 0.8772
Epoch 3/400
444/444 [==============================] - ETA: 0s - loss: 0.4632 - table_output_loss: 0.2349 - column_output_loss: 0.2284 - table_output_accuracy: 0.9041 - column_output_accuracy: 0.8903
Epoch 3: val_loss did not improve from 0.49438
444/444 [==============================] - 17s 39ms/step - loss: 0.4632 - table_output_loss: 0.2349 - column_output_loss: 0.2284 - table_output_accuracy: 0.9041 - column_output_accuracy: 0.8903 - val_loss: 0.6988 - val_table_output_loss: 0.3792 - val_column_output_loss: 0.3196 - val_table_output_accuracy: 0.8548 - val_column_output_accuracy: 0.8593
Epoch 4/400
443/444 [============================>.

444/444 [==============================] - 22s 50ms/step - loss: 0.4088 - table_output_loss: 0.2049 - column_output_loss: 0.2039 - table_output_accuracy: 0.9132 - column_output_accuracy: 0.8963 - val_loss: 0.3919 - val_table_output_loss: 0.1814 - val_column_output_loss: 0.2106 - val_table_output_accuracy: 0.9247 - val_column_output_accuracy: 0.8965
Epoch 6/400
444/444 [==============================] - ETA: 0s - loss: 0.4061 - table_output_loss: 0.2009 - column_output_loss: 0.2052 - table_output_accuracy: 0.9142 - column_output_accuracy: 0.8963
Epoch 6: val_loss improved from 0.39195 to 0.38955, saving model to /content/drive/MyDrive/Colab Notebooks/Model_output/mymodel_6


444/444 [==============================] - 22s 48ms/step - loss: 0.4061 - table_output_loss: 0.2009 - column_output_loss: 0.2052 - table_output_accuracy: 0.9142 - column_output_accuracy: 0.8963 - val_loss: 0.3895 - val_table_output_loss: 0.1836 - val_column_output_loss: 0.2059 - val_table_output_accuracy: 0.9248 - val_column_output_accuracy: 0.8985
Epoch 7/400
444/444 [==============================] - ETA: 0s - loss: 0.3872 - table_output_loss: 0.1925 - column_output_loss: 0.1947 - table_output_accuracy: 0.9176 - column_output_accuracy: 0.8994
Epoch 7: val_loss did not improve from 0.38955
444/444 [==============================] - 17s 39ms/step - loss: 0.3872 - table_output_loss: 0.1925 - column_output_loss: 0.1947 - table_output_accuracy: 0.9176 - column_output_accuracy: 0.8994 - val_loss: 0.4809 - val_table_output_loss: 0.1758 - val_column_output_loss: 0.3051 - val_table_output_accuracy: 0.9286 - val_column_output_accuracy: 0.8650
Epoch 8/400
443/444 [============================>.

...

444/444 [==============================] - 22s 49ms/step - loss: 0.2618 - table_output_loss: 0.1234 - column_output_loss: 0.1384 - table_output_accuracy: 0.9416 - column_output_accuracy: 0.9178 - val_loss: 0.2847 - val_table_output_loss: 0.1349 - val_column_output_loss: 0.1498 - val_table_output_accuracy: 0.9420 - val_column_output_accuracy: 0.9138
Epoch 138/400
443/444 [============================>.] - ETA: 0s - loss: 0.2548 - table_output_loss: 0.1184 - column_output_loss: 0.1364 - table_output_accuracy: 0.9431 - column_output_accuracy: 0.9183
Epoch 138: val_loss did not improve from 0.28468
444/444 [==============================] - 17s 38ms/step - loss: 0.2548 - table_output_loss: 0.1184 - column_output_loss: 0.1364 - table_output_accuracy: 0.9431 - column_output_accuracy: 0.9182 - val_loss: 0.3716 - val_table_output_loss: 0.1879 - val_column_output_loss: 0.1837 - val_table_output_accuracy: 0.9251 - val_column_output_accuracy: 0.9040
Epoch 139/400
443/444 [========================

444/444 [==============================] - 22s 50ms/step - loss: 0.2685 - table_output_loss: 0.1240 - column_output_loss: 0.1445 - table_output_accuracy: 0.9413 - column_output_accuracy: 0.9163 - val_loss: 0.2818 - val_table_output_loss: 0.1394 - val_column_output_loss: 0.1424 - val_table_output_accuracy: 0.9415 - val_column_output_accuracy: 0.9180
Epoch 140/400
443/444 [============================>.] - ETA: 0s - loss: 0.2659 - table_output_loss: 0.1238 - column_output_loss: 0.1421 - table_output_accuracy: 0.9406 - column_output_accuracy: 0.9162
Epoch 140: val_loss did not improve from 0.28183
444/444 [==============================] - 17s 38ms/step - loss: 0.2663 - table_output_loss: 0.1240 - column_output_loss: 0.1423 - table_output_accuracy: 0.9405 - column_output_accuracy: 0.9160 - val_loss: 0.3721 - val_table_output_loss: 0.1920 - val_column_output_loss: 0.1801 - val_table_output_accuracy: 0.9261 - val_column_output_accuracy: 0.9084
Epoch 141/400
443/444 [========================

444/444 [==============================] - 22s 50ms/step - loss: 0.2500 - table_output_loss: 0.1153 - column_output_loss: 0.1347 - table_output_accuracy: 0.9443 - column_output_accuracy: 0.9190 - val_loss: 0.2613 - val_table_output_loss: 0.1195 - val_column_output_loss: 0.1418 - val_table_output_accuracy: 0.9467 - val_column_output_accuracy: 0.9182
Epoch 186/400
444/444 [==============================] - ETA: 0s - loss: 0.2453 - table_output_loss: 0.1134 - column_output_loss: 0.1320 - table_output_accuracy: 0.9451 - column_output_accuracy: 0.9200
Epoch 186: val_loss did not improve from 0.26125
444/444 [==============================] - 17s 38ms/step - loss: 0.2453 - table_output_loss: 0.1134 - column_output_loss: 0.1320 - table_output_accuracy: 0.9451 - column_output_accuracy: 0.9200 - val_loss: 0.5700 - val_table_output_loss: 0.2642 - val_column_output_loss: 0.3058 - val_table_output_accuracy: 0.9045 - val_column_output_accuracy: 0.8731
Epoch 187/400
443/444 [========================

444/444 [==============================] - 22s 51ms/step - loss: 0.2370 - table_output_loss: 0.1080 - column_output_loss: 0.1289 - table_output_accuracy: 0.9467 - column_output_accuracy: 0.9216 - val_loss: 0.2526 - val_table_output_loss: 0.1187 - val_column_output_loss: 0.1339 - val_table_output_accuracy: 0.9499 - val_column_output_accuracy: 0.9219
Epoch 222/400
443/444 [============================>.] - ETA: 0s - loss: 0.2381 - table_output_loss: 0.1098 - column_output_loss: 0.1282 - table_output_accuracy: 0.9462 - column_output_accuracy: 0.9216
Epoch 222: val_loss did not improve from 0.25264
444/444 [==============================] - 17s 38ms/step - loss: 0.2383 - table_output_loss: 0.1099 - column_output_loss: 0.1284 - table_output_accuracy: 0.9461 - column_output_accuracy: 0.9215 - val_loss: 0.3076 - val_table_output_loss: 0.1539 - val_column_output_loss: 0.1537 - val_table_output_accuracy: 0.9368 - val_column_output_accuracy: 0.9162
Epoch 223/400
444/444 [========================

444/444 [==============================] - 21s 48ms/step - loss: 0.2386 - table_output_loss: 0.1083 - column_output_loss: 0.1303 - table_output_accuracy: 0.9468 - column_output_accuracy: 0.9210 - val_loss: 0.2501 - val_table_output_loss: 0.1187 - val_column_output_loss: 0.1314 - val_table_output_accuracy: 0.9470 - val_column_output_accuracy: 0.9206
Epoch 247/400
443/444 [============================>.] - ETA: 0s - loss: 0.2369 - table_output_loss: 0.1097 - column_output_loss: 0.1273 - table_output_accuracy: 0.9470 - column_output_accuracy: 0.9223
Epoch 247: val_loss did not improve from 0.25006
444/444 [==============================] - 17s 38ms/step - loss: 0.2366 - table_output_loss: 0.1095 - column_output_loss: 0.1271 - table_output_accuracy: 0.9471 - column_output_accuracy: 0.9224 - val_loss: 0.3053 - val_table_output_loss: 0.1527 - val_column_output_loss: 0.1526 - val_table_output_accuracy: 0.9384 - val_column_output_accuracy: 0.9194
Epoch 248/400
443/444 [========================